# MAK DRAFT HELPER 2019

In [1381]:
import pandas as pd
import numpy as np

## Batters

In [1382]:
batters = pd.read_csv('./FantasyPros_2019_Projections_H.csv')

Fantasy Pros (fantasypros.com) produces an aggregated expert projections for each player across several key stats. No need to reinvent the wheel like Nate Silver did in his book with inefficient models - we just take what the experts project (using their models) and look at their consensus.

In [1383]:
batters.head()

,Player,Team,Positions,AB,R,HR,RBI,SB,AVG,OBP,H,2B,3B,BB,SO,SLG,OPS,Own
0,Mike Trout,LAA,"CF,DH",509.0,112.0,38.0,95.0,23.0,0.307,0.438,156.0,28.0,4.0,119.0,127.0,0.606,1.044,99%
1,Mookie Betts,BOS,"CF,RF",586.0,115.0,30.0,91.0,26.0,0.305,0.385,179.0,43.0,4.0,76.0,93.0,0.544,0.929,99%
2,Jose Ramirez,CLE,"2B,3B",567.0,100.0,30.0,96.0,24.0,0.286,0.374,162.0,41.0,4.0,80.0,76.0,0.534,0.908,99%
3,Nolan Arenado,COL,3B,584.0,98.0,37.0,111.0,2.0,0.291,0.362,170.0,38.0,4.0,65.0,115.0,0.559,0.921,99%
4,J.D. Martinez,BOS,"LF,RF,DH",543.0,97.0,39.0,114.0,4.0,0.303,0.376,164.0,33.0,2.0,63.0,148.0,0.587,0.962,99%


In [1384]:
del batters['OBP']
del batters['2B']
del batters['3B']
del batters['SLG']
del batters['OPS']

In [1385]:
batters = batters[batters['AB'] >= 200]

In [1386]:
batters['BB/KO'] = batters['BB'] / batters['SO']

For BB/KO, we want to look at the number of walks ABOVE the average BB/KO of the selection set (which is all batters projected to have over 200 AB's). This will approproately weight the average based on projected Fantasy production. We do the same with batting average.

In [1387]:
bbko_mean = batters['BB/KO'].mean()
bbko_mean

0.3989026493701326

In [1388]:
batters['BB+'] = (batters['BB/KO'] - bbko_mean) * batters['SO']

In [1389]:
batters['AVG'] = batters['H'] / batters['AB']

In [1390]:
avg_mean = batters['AVG'].mean()
avg_mean

0.2530256240141296

In [1391]:
batters['AVG+'] = (batters['AVG'] - avg_mean) * batters['AB']

For this year, I'm simply standardizing each metric to the 0-100 linear scale with 100 being the top performer for that metric and 0 being the bottom. Since the scales are different for each one, this makes sense. In the past, I've used Z scores, but it is easier to be able to mentally calculate the actual metric in some cases. It also makes sense since these stats all accumulate over the course of the season/week.

In [1392]:
batters['AVGS'] = round( ( batters['AVG+'] - batters['AVG+'].min() ) / ( batters['AVG+'].max() - batters['AVG+'].min() ) * 100 )
batters['BBKOS'] = round( ( batters['BB+'] - batters['BB+'].min() ) / ( batters['BB+'].max() - batters['BB+'].min() ) * 100 )
batters['HRS'] = round( ( batters['HR'] - batters['HR'].min() ) / ( batters['HR'].max() - batters['HR'].min() ) * 100 )
batters['RS'] = round( ( batters['R'] - batters['R'].min() ) / ( batters['R'].max() - batters['R'].min() ) * 100 )
batters['RBIS'] = round( ( batters['RBI'] - batters['RBI'].min() ) / ( batters['RBI'].max() - batters['RBI'].min() ) * 100 )
batters['SBS'] = round( ( batters['SB'] - batters['SB'].min() ) / ( batters['SB'].max() - batters['SB'].min() ) * 100 )
batters['SCORE'] = batters['AVGS'] + batters['BBKOS'] + batters['HRS'] + batters['RS'] + batters['RBIS'] + batters['SBS']
batters['RANK'] = batters['SCORE'].rank(method='average', na_option='keep', ascending=False, pct=False)

In [1393]:
batters.sort_values(['SCORE'], ascending=[0])[0:5]

,Player,Team,Positions,AB,R,HR,RBI,SB,AVG,H,...,BB+,AVG+,AVGS,BBKOS,HRS,RS,RBIS,SBS,SCORE,RANK
0,Mike Trout,LAA,"CF,DH",509.0,112.0,38.0,95.0,23.0,0.306483,156.0,...,68.339364,27.209957,86.0,100.0,90.0,97.0,81.0,51.0,505.0,1.0
1,Mookie Betts,BOS,"CF,RF",586.0,115.0,30.0,91.0,26.0,0.305461,179.0,...,38.902054,30.726984,91.0,77.0,71.0,100.0,77.0,58.0,474.0,2.0
2,Jose Ramirez,CLE,"2B,3B",567.0,100.0,30.0,96.0,24.0,0.285714,162.0,...,49.683399,18.534471,71.0,86.0,71.0,84.0,82.0,53.0,447.0,3.0
4,J.D. Martinez,BOS,"LF,RF,DH",543.0,97.0,39.0,114.0,4.0,0.302026,164.0,...,3.962408,26.607086,85.0,50.0,93.0,81.0,100.0,9.0,418.0,4.0
9,Bryce Harper,PHI,"CF,RF",517.0,98.0,34.0,99.0,11.0,0.266925,138.0,...,52.164603,7.185752,53.0,88.0,80.0,82.0,85.0,24.0,412.0,5.0


## Pitchers

In [1394]:
pitchers = pd.read_csv('./FantasyPros_2019_Projections_P.csv')

In [1395]:
pitchers.head()

,Player,Team,Positions,IP,K,W,SV,ERA,WHIP,ER,H,BB,HR,G,GS,L,CG,Own
0,Max Scherzer,WSH,SP,206.9,269.0,16.0,0.0,2.94,1.01,68.0,154.0,55.0,25.0,32.0,32.0,8.0,2.0,99%
1,Chris Sale,BOS,SP,186.0,249.0,15.0,0.0,2.68,0.97,55.0,141.0,40.0,19.0,30.0,30.0,6.0,0.0,99%
2,Jacob deGrom,NYM,SP,207.6,248.0,15.0,0.0,2.75,1.05,63.0,166.0,52.0,20.0,32.0,32.0,9.0,1.0,99%
3,Justin Verlander,HOU,SP,201.1,245.0,16.0,0.0,3.17,1.05,71.0,159.0,51.0,27.0,32.0,31.0,8.0,1.0,99%
4,Corey Kluber,CLE,SP,206.9,221.0,16.0,0.0,3.23,1.07,74.0,180.0,43.0,24.0,32.0,32.0,8.0,3.0,99%


We score on Holds+SVS, so I need to bring that stat in from elsewhere

In [1396]:
holds = pd.read_csv('./cbs_P_Scoring.csv', header=1)

In [1397]:
holds.head()

,Avail,Player,MLB,Eligible,INN,ERA,WHIP,K,S,HD,W-L,BB,H,ER,Rank,Unnamed: 15
0,Low hanging fruit,Max Scherzer SP | WAS,0.0,P,206.3,2.97,1.05,255.0,0.0,0.0,10.0,57.0,159.0,68.0,7.0,NaN
1,Team Gorgeous,Jacob deGrom SP | NYM,0.0,P,197.3,2.46,1.09,221.0,0.0,0.0,6.0,51.0,164.0,54.0,14.0,NaN
2,Rakuten Golden Eagles,Chris Sale SP | BOS,0.0,P,166.7,2.70,1.05,214.0,0.0,0.0,9.0,38.0,137.0,50.0,15.0,NaN
3,stroke me strokeme,Justin Verlander SP | HOU,0.0,P,198.3,3.04,1.11,245.0,0.0,0.0,11.0,58.0,163.0,67.0,20.0,NaN
4,Tommy's Shinebox,Corey Kluber SP | CLE,0.0,P,205.0,3.29,1.14,229.0,0.0,0.0,12.0,47.0,187.0,75.0,22.0,NaN


In [1398]:
holds = holds[ holds['HD'] > 0 ]
holds['Name'] = holds['Player']
holds['Player'] = holds['Name'].str.extract(r'^(.+ .+) \w+ \|', expand=False)
holds['Team'] = holds['Name'].str.extract(r'\|\s(\w\w\w|\w\w)(?:\s|$)', expand=False)
holds = holds[ ['Player', 'Team', 'HD'] ]
holds = holds.replace('WAS', 'WSH')
holds = holds.replace('CHW', 'CWS')
holds.head()

,Player,Team,HD
14,Edwin Diaz,NYM,3.0
16,Blake Treinen,OAK,3.0
18,Craig Kimbrel,BOS,3.0
22,Aroldis Chapman,NYY,3.0
28,Kenley Jansen,LAD,3.0


In [1399]:
pitchers = pitchers.merge(holds, on=['Player', 'Team'], how='left')

In [1400]:
pitchers[pitchers['Player'].isnull()].head()

,Player,Team,Positions,IP,K,W,SV,ERA,WHIP,ER,H,BB,HR,G,GS,L,CG,Own,HD
883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
884,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1401]:
del pitchers['HR']
del pitchers['G']
del pitchers['GS']
del pitchers['CG']

In [1402]:
pitchers = pitchers[pitchers['IP'] >= 40]

In [1403]:
pitchers['ERA'] = pitchers['ER'] / pitchers['IP'] * 9
era_mean = pitchers['ERA'].mean()
pitchers['ERA+'] = ( era_mean - pitchers['ERA'] ) * pitchers['IP']

In [1404]:
pitchers['WHIP'] = (pitchers['BB'] + pitchers['H']) / pitchers['IP']
whip_mean = pitchers['WHIP'].mean()
pitchers['WHIP+'] = ( whip_mean -  pitchers['WHIP'] ) * pitchers['IP']

In [1405]:
pitchers['W-L'] = pitchers['W'] - pitchers['L']

In [1406]:
pitchers['HD'].fillna(0, inplace=True)
pitchers['SV+HD'] = pitchers['SV'] + pitchers['HD']

In [1407]:
pitchers['ERAS'] = round( ( pitchers['ERA+'] - pitchers['ERA+'].min() ) / ( pitchers['ERA+'].max() - pitchers['ERA+'].min() ) * 100 )
pitchers['WHIPS'] = round( ( pitchers['WHIP+'] - pitchers['WHIP+'].min() ) / ( pitchers['WHIP+'].max() - pitchers['WHIP+'].min() ) * 100 )
pitchers['KS'] = round( ( pitchers['K'] - pitchers['K'].min() ) / ( pitchers['K'].max() - pitchers['K'].min() ) * 100 )
pitchers['SV+HDS'] = round( ( pitchers['SV+HD'] - pitchers['SV+HD'].min() ) / ( pitchers['SV+HD'].max() - pitchers['SV+HD'].min() ) * 100 )
pitchers['WLS'] = round( ( pitchers['W-L'] - pitchers['W-L'].min() ) / ( pitchers['W-L'].max() - pitchers['W-L'].min() ) * 100 )
pitchers['SCORE'] = pitchers['ERAS'] + pitchers['WHIPS'] + pitchers['KS'] + pitchers['SV+HDS'] + pitchers['WLS']
pitchers['RANK'] = pitchers['SCORE'].rank(method='average', na_option='keep', ascending=False, pct=False)

In [1408]:
pitchers.sort_values(['SCORE'], ascending=[0])[0:5]

,Player,Team,Positions,IP,K,W,SV,ERA,WHIP,ER,...,WHIP+,W-L,SV+HD,ERAS,WHIPS,KS,SV+HDS,WLS,SCORE,RANK
1,Chris Sale,BOS,SP,186.0,249.0,15.0,0.0,2.661290,0.973118,55.0,...,74.788030,9.0,0.0,97.0,99.0,92.0,0.0,100.0,388.0,1.0
0,Max Scherzer,WSH,SP,206.9,269.0,16.0,0.0,2.957951,1.010150,68.0,...,75.529803,8.0,0.0,92.0,100.0,100.0,0.0,93.0,385.0,2.0
2,Jacob deGrom,NYM,SP,207.6,248.0,15.0,0.0,2.731214,1.050096,63.0,...,67.492446,6.0,0.0,100.0,94.0,91.0,0.0,79.0,364.0,3.0
3,Justin Verlander,HOU,SP,201.1,245.0,16.0,0.0,3.177524,1.044257,71.0,...,66.553617,8.0,0.0,84.0,94.0,90.0,0.0,93.0,361.0,4.0
4,Corey Kluber,CLE,SP,206.9,221.0,16.0,0.0,3.218946,1.077815,74.0,...,61.529803,8.0,0.0,84.0,90.0,80.0,0.0,93.0,347.0,5.0


## Combining Pitchers and Batters

Since we score across five pitching and six hitting categories, with the weekly winners determined by the highest aggregate scores in each category, it makes sense to compare score weight of batters vs. pitchers 

For the last couple of seasons, I've tried to "break the game" by focusing only on all the pitching categories plus two batting categories (AVG and BBKO) where there are batters generally available with high performance at a low cost. This has had mixed results, as the league adjusted accordingly.

In [1409]:
batters_lim = batters[ ['Player', 'Team', 'Positions', 'AVGS', 'BBKOS', 'HRS', 'RS', 'RBIS', 'SBS', 'SCORE'] ]
pitchers_lim = pitchers[ ['Player', 'Team', 'Positions', 'ERAS', 'WHIPS', 'KS', 'SV+HDS', 'WLS', 'SCORE'] ]
all_players = pd.concat([batters_lim, pitchers_lim],sort=True)
all_players['RANK'] = all_players['SCORE'].rank(method='average', na_option='keep', ascending=False, pct=False)
all_players = all_players.sort_values(['SCORE'], ascending=[0])

In [1410]:
all_players.head(25)

,AVGS,BBKOS,ERAS,HRS,KS,Player,Positions,RBIS,RS,SBS,SCORE,SV+HDS,Team,WHIPS,WLS,RANK
0,86.0,100.0,NaN,90.0,NaN,Mike Trout,"CF,DH",81.0,97.0,51.0,505.0,NaN,LAA,NaN,NaN,1.0
1,91.0,77.0,NaN,71.0,NaN,Mookie Betts,"CF,RF",77.0,100.0,58.0,474.0,NaN,BOS,NaN,NaN,2.0
2,71.0,86.0,NaN,71.0,NaN,Jose Ramirez,"2B,3B",82.0,84.0,53.0,447.0,NaN,CLE,NaN,NaN,3.0
4,85.0,50.0,NaN,93.0,NaN,J.D. Martinez,"LF,RF,DH",100.0,81.0,9.0,418.0,NaN,BOS,NaN,NaN,4.0
9,53.0,88.0,NaN,80.0,NaN,Bryce Harper,"CF,RF",85.0,82.0,24.0,412.0,NaN,PHI,NaN,NaN,5.0
3,77.0,62.0,NaN,88.0,NaN,Nolan Arenado,3B,97.0,82.0,4.0,410.0,NaN,COL,NaN,NaN,6.5
5,85.0,61.0,NaN,63.0,NaN,Christian Yelich,"LF,CF,RF",78.0,85.0,38.0,410.0,NaN,MIL,NaN,NaN,6.5
11,71.0,80.0,NaN,63.0,NaN,Alex Bregman,"3B,SS",78.0,82.0,24.0,398.0,NaN,HOU,NaN,NaN,8.0
8,69.0,66.0,NaN,68.0,NaN,Francisco Lindor,SS,69.0,85.0,40.0,397.0,NaN,CLE,NaN,NaN,9.0
6,68.0,52.0,NaN,41.0,NaN,Trea Turner,SS,55.0,79.0,100.0,395.0,NaN,WSH,NaN,NaN,10.5


## Join on list of currently drafted players

In [1411]:
draft = pd.read_csv('./results.csv', header=1)
draft.head()

,Pick,Team,Player,Elapsed Time,Unnamed: 4
0,1,MitchMoreland'sBattingAverage,Mike Trout CF | LAA,NaN,NaN
1,2,Tommy's Shinebox,Mookie Betts RF | BOS,8 hrs 29 min 27 sec,NaN
2,3,La Flama Blanca,Jose Altuve 2B | HOU,32 min 24 sec,NaN
3,4,Low hanging fruit,Max Scherzer SP | WAS,2 min 59 sec,NaN
4,5,Foul_Tip,Nolan Arenado 3B | COL,57 min 17 sec,NaN


In [1412]:
draft = draft[ ['Player', 'Team'] ]
draft['Name'] = draft['Player']
draft['Owner'] = draft['Team']
draft = draft[ draft['Player'] != 'Player' ]
draft['Player'] = draft['Name'].str.extract(r'^(.+ .+) \w+ \|', expand=False)
draft['Team'] = draft['Name'].str.extract(r'\|\s(\w\w\w|\w\w)(?:\s|$)', expand=False)
draft = draft.dropna()

In [1413]:
draft = draft.replace('WAS', 'WSH')
draft = draft.replace('CHW', 'CWS')

In [1414]:
draft.head()

,Player,Team,Name,Owner
0,Mike Trout,LAA,Mike Trout CF | LAA,MitchMoreland'sBattingAverage
1,Mookie Betts,BOS,Mookie Betts RF | BOS,Tommy's Shinebox
2,Jose Altuve,HOU,Jose Altuve 2B | HOU,La Flama Blanca
3,Max Scherzer,WSH,Max Scherzer SP | WAS,Low hanging fruit
4,Nolan Arenado,COL,Nolan Arenado 3B | COL,Foul_Tip


In [1415]:
draft.loc[draft.Player == 'Craig Kimbrel', 'Team'] = np.nan
all_players_owners = all_players.merge(draft, on=['Player', 'Team'], how='outer')

In [1416]:
all_players_owners.head()

,AVGS,BBKOS,ERAS,HRS,KS,Player,Positions,RBIS,RS,SBS,SCORE,SV+HDS,Team,WHIPS,WLS,RANK,Name,Owner
0,86.0,100.0,NaN,90.0,NaN,Mike Trout,"CF,DH",81.0,97.0,51.0,505.0,NaN,LAA,NaN,NaN,1.0,Mike Trout CF | LAA,MitchMoreland'sBattingAverage
1,91.0,77.0,NaN,71.0,NaN,Mookie Betts,"CF,RF",77.0,100.0,58.0,474.0,NaN,BOS,NaN,NaN,2.0,Mookie Betts RF | BOS,Tommy's Shinebox
2,71.0,86.0,NaN,71.0,NaN,Jose Ramirez,"2B,3B",82.0,84.0,53.0,447.0,NaN,CLE,NaN,NaN,3.0,Jose Ramirez 3B | CLE (Keeper),BC-Mizusawa South Monkeyz
3,85.0,50.0,NaN,93.0,NaN,J.D. Martinez,"LF,RF,DH",100.0,81.0,9.0,418.0,NaN,BOS,NaN,NaN,4.0,J.D. Martinez DH | BOS,stroke me strokeme
4,53.0,88.0,NaN,80.0,NaN,Bryce Harper,"CF,RF",85.0,82.0,24.0,412.0,NaN,PHI,NaN,NaN,5.0,Bryce Harper RF | PHI,Regular Rubens


Ensure all players merged. Cross fingers that there aren't any players on the same team with the same name. That would be weird though. This takes advatage of the characterisitics of an outer join.

In [1417]:
all_players_owners[all_players_owners['Player'].isnull()].head()

,AVGS,BBKOS,ERAS,HRS,KS,Player,Positions,RBIS,RS,SBS,SCORE,SV+HDS,Team,WHIPS,WLS,RANK,Name,Owner


In [1418]:
all_players_owners[all_players_owners['Positions'].isnull()].head()

,AVGS,BBKOS,ERAS,HRS,KS,Player,Positions,RBIS,RS,SBS,SCORE,SV+HDS,Team,WHIPS,WLS,RANK,Name,Owner


## All Currently Undrafted

In [1419]:
all_players_owners = all_players_owners[ ['Player', 'Positions', 'Team', 'SCORE', 'RANK', 'AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'ERAS', 'WHIPS', 'KS', 'WLS', 'SV+HDS', 'Owner'] ]
#Manually add minors drafted
all_players_owners.loc[all_players_owners.Player == 'Vladimir Guerrero Jr.', 'Owner'] = 'Low hanging fruit'
free_agents = all_players_owners[all_players_owners['Owner'].isnull()]
del free_agents['Owner']
free_agents.head(50)

,Player,Positions,Team,SCORE,RANK,AVGS,BBKOS,HRS,RBIS,RS,SBS,ERAS,WHIPS,KS,WLS,SV+HDS
95,Andrelton Simmons,SS,LAA,285.0,96.5,67.0,61.0,24.0,54.0,52.0,27.0,NaN,NaN,NaN,NaN,NaN
96,Josh Bell,1B,PIT,285.0,96.5,51.0,68.0,41.0,61.0,57.0,7.0,NaN,NaN,NaN,NaN,NaN
101,Seranthony Dominguez,"SP,RP",PHI,281.0,102.5,NaN,NaN,NaN,NaN,NaN,NaN,59.0,57.0,24.0,43.0,98.0
103,Andrew Miller,RP,STL,281.0,102.5,NaN,NaN,NaN,NaN,NaN,NaN,59.0,57.0,22.0,43.0,100.0
105,Shohei Ohtani,"SP,DH",LAA,279.0,106.5,56.0,44.0,56.0,58.0,45.0,20.0,NaN,NaN,NaN,NaN,NaN
108,Ryan Braun,"1B,LF",MIL,278.0,110.0,51.0,47.0,49.0,56.0,48.0,27.0,NaN,NaN,NaN,NaN,NaN
110,Jorge Polanco,SS,MIN,278.0,110.0,57.0,50.0,32.0,54.0,56.0,29.0,NaN,NaN,NaN,NaN,NaN
111,Victor Robles,RF,WSH,277.0,112.0,54.0,47.0,27.0,42.0,49.0,58.0,NaN,NaN,NaN,NaN,NaN
113,Justin Smoak,"1B,DH",TOR,276.0,114.0,33.0,58.0,63.0,65.0,55.0,2.0,NaN,NaN,NaN,NaN,NaN
116,Shin-Soo Choo,"LF,RF,DH",TEX,275.0,116.5,44.0,63.0,44.0,47.0,61.0,16.0,NaN,NaN,NaN,NaN,NaN


## Positions

### 1B

In [1420]:
first_base = all_players_owners[all_players_owners['Positions'].str.contains('1B')]
first_base = first_base[ ['Player', 'Positions', 'Team', 'SCORE', 'RANK', 'AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'Owner'] ]

In [1421]:
first_base.head(25)

,Player,Positions,Team,SCORE,RANK,AVGS,BBKOS,HRS,RBIS,RS,SBS,Owner
14,Paul Goldschmidt,1B,STL,384.0,15.0,68.0,68.0,68.0,79.0,79.0,22.0,Martin Awaiting Klub
16,Freddie Freeman,1B,ATL,382.0,17.0,84.0,66.0,61.0,78.0,75.0,18.0,Walk Off
17,Anthony Rizzo,1B,CHC,379.0,19.5,65.0,79.0,66.0,84.0,69.0,16.0,Low hanging fruit
23,Rhys Hoskins,"1B,LF",PHI,365.0,24.0,38.0,67.0,85.0,89.0,75.0,11.0,La Flama Blanca
27,Joey Votto,1B,CIN,360.0,28.0,72.0,98.0,49.0,66.0,66.0,9.0,Rakuten Golden Eagles
29,Cody Bellinger,"1B,CF",LAD,351.0,30.0,47.0,55.0,73.0,78.0,71.0,27.0,Foul_Tip
37,Whit Merrifield,"1B,2B,CF,RF,DH",KC,339.0,38.0,71.0,51.0,29.0,50.0,65.0,73.0,Regular Rubens
39,Matt Carpenter,"1B,2B,3B",STL,331.0,40.0,41.0,77.0,66.0,61.0,79.0,7.0,Regular Rubens
43,Daniel Murphy,"1B,2B",COL,326.0,43.5,86.0,56.0,49.0,65.0,61.0,9.0,Martin Awaiting Klub
47,Carlos Santana,"1B,3B",CLE,324.0,47.0,37.0,92.0,54.0,70.0,64.0,7.0,Tommy's Shinebox


### 2B

In [1422]:
second_base = all_players_owners[all_players_owners['Positions'].str.contains('2B')]
second_base = second_base[ ['Player', 'Positions', 'Team', 'SCORE', 'RANK', 'AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'Owner'] ]

In [1423]:
second_base.head(25)

,Player,Positions,Team,SCORE,RANK,AVGS,BBKOS,HRS,RBIS,RS,SBS,Owner
2,Jose Ramirez,"2B,3B",CLE,447.0,3.0,71.0,86.0,71.0,82.0,84.0,53.0,BC-Mizusawa South Monkeyz
10,Jose Altuve,2B,HOU,395.0,10.5,100.0,65.0,39.0,64.0,78.0,49.0,La Flama Blanca
31,Javier Baez,"2B,3B,SS",CHC,347.0,32.5,63.0,23.0,71.0,81.0,71.0,38.0,Team Gorgeous
37,Whit Merrifield,"1B,2B,CF,RF,DH",KC,339.0,38.0,71.0,51.0,29.0,50.0,65.0,73.0,Regular Rubens
39,Matt Carpenter,"1B,2B,3B",STL,331.0,40.0,41.0,77.0,66.0,61.0,79.0,7.0,Regular Rubens
43,Daniel Murphy,"1B,2B",COL,326.0,43.5,86.0,56.0,49.0,65.0,61.0,9.0,Martin Awaiting Klub
55,Ozzie Albies,2B,ATL,314.0,56.0,57.0,45.0,46.0,59.0,71.0,36.0,Walk Off
56,Travis Shaw,"1B,3B,2B",MIL,311.0,59.5,42.0,60.0,66.0,72.0,58.0,13.0,Team Gorgeous
63,Adalberto Mondesi,"2B,SS",KC,309.0,64.5,39.0,23.0,44.0,51.0,59.0,93.0,Foul_Tip
77,Scooter Gennett,2B,CIN,299.0,78.0,66.0,39.0,51.0,70.0,62.0,11.0,MitchMoreland'sBattingAverage


### SS

In [1424]:
shortstop = all_players_owners[all_players_owners['Positions'].str.contains('SS')]
shortstop = shortstop[ ['Player', 'Positions', 'Team', 'SCORE', 'RANK', 'AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'Owner'] ]

In [1425]:
shortstop.head(25)

,Player,Positions,Team,SCORE,RANK,AVGS,BBKOS,HRS,RBIS,RS,SBS,Owner
7,Alex Bregman,"3B,SS",HOU,398.0,8.0,71.0,80.0,63.0,78.0,82.0,24.0,Martin Awaiting Klub
8,Francisco Lindor,SS,CLE,397.0,9.0,69.0,66.0,68.0,69.0,85.0,40.0,Walk Off
9,Trea Turner,SS,WSH,395.0,10.5,68.0,52.0,41.0,55.0,79.0,100.0,Martin Awaiting Klub
18,Manny Machado,"3B,SS",SD,379.0,19.5,70.0,62.0,76.0,80.0,69.0,22.0,MitchMoreland'sBattingAverage
26,Trevor Story,SS,COL,361.0,26.5,57.0,32.0,78.0,84.0,68.0,42.0,Team Gorgeous
31,Javier Baez,"2B,3B,SS",CHC,347.0,32.5,63.0,23.0,71.0,81.0,71.0,38.0,Team Gorgeous
35,Xander Bogaerts,SS,BOS,342.0,36.0,70.0,57.0,46.0,76.0,71.0,22.0,BC-Mizusawa South Monkeyz
44,Carlos Correa,SS,HOU,325.0,45.0,56.0,60.0,54.0,77.0,67.0,11.0,BC-Mizusawa South Monkeyz
45,Jean Segura,SS,PHI,324.0,47.0,77.0,49.0,29.0,50.0,72.0,47.0,Low hanging fruit
63,Adalberto Mondesi,"2B,SS",KC,309.0,64.5,39.0,23.0,44.0,51.0,59.0,93.0,Foul_Tip


### 3B

In [1426]:
third_base = all_players_owners[all_players_owners['Positions'].str.contains('3B')]
third_base = third_base[ ['Player', 'Positions', 'Team', 'SCORE', 'RANK', 'AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'Owner'] ]

In [1427]:
third_base.head(25)

,Player,Positions,Team,SCORE,RANK,AVGS,BBKOS,HRS,RBIS,RS,SBS,Owner
2,Jose Ramirez,"2B,3B",CLE,447.0,3.0,71.0,86.0,71.0,82.0,84.0,53.0,BC-Mizusawa South Monkeyz
5,Nolan Arenado,3B,COL,410.0,6.5,77.0,62.0,88.0,97.0,82.0,4.0,Foul_Tip
7,Alex Bregman,"3B,SS",HOU,398.0,8.0,71.0,80.0,63.0,78.0,82.0,24.0,Martin Awaiting Klub
18,Manny Machado,"3B,SS",SD,379.0,19.5,70.0,62.0,76.0,80.0,69.0,22.0,MitchMoreland'sBattingAverage
28,Anthony Rendon,3B,WSH,358.0,29.0,77.0,70.0,54.0,77.0,69.0,11.0,stroke me strokeme
30,Kris Bryant,"3B,RF",CHC,350.0,31.0,62.0,63.0,63.0,70.0,79.0,13.0,Rakuten Golden Eagles
31,Javier Baez,"2B,3B,SS",CHC,347.0,32.5,63.0,23.0,71.0,81.0,71.0,38.0,Team Gorgeous
39,Matt Carpenter,"1B,2B,3B",STL,331.0,40.0,41.0,77.0,66.0,61.0,79.0,7.0,Regular Rubens
40,Eugenio Suarez,3B,CIN,328.0,41.0,53.0,53.0,71.0,80.0,64.0,7.0,Low hanging fruit
47,Carlos Santana,"1B,3B",CLE,324.0,47.0,37.0,92.0,54.0,70.0,64.0,7.0,Tommy's Shinebox


### C

In [1428]:
catcher = all_players_owners[all_players_owners['Positions'].str.contains('^C$|C,', regex=True)]
catcher = catcher[ ['Player', 'Positions', 'Team', 'SCORE', 'RANK', 'AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'Owner'] ]

In [1429]:
catcher.head(25)

,Player,Positions,Team,SCORE,RANK,AVGS,BBKOS,HRS,RBIS,RS,SBS,Owner
100,J.T. Realmuto,"C,1B",PHI,281.0,102.5,59.0,43.0,51.0,62.0,55.0,11.0,Team Gorgeous
119,Gary Sanchez,"C,DH",NYY,273.0,120.0,33.0,46.0,68.0,67.0,55.0,4.0,La Flama Blanca
171,Buster Posey,"C,1B",SF,248.0,173.0,67.0,67.0,22.0,43.0,40.0,9.0,MitchMoreland'sBattingAverage
222,Yasmani Grandal,C,MIL,232.0,221.0,34.0,57.0,51.0,49.0,39.0,2.0,Martin Awaiting Klub
225,Yadier Molina,C,STL,231.0,225.0,52.0,49.0,34.0,50.0,35.0,11.0,Walk Off
251,Willson Contreras,C,CHC,223.0,250.0,44.0,51.0,34.0,48.0,37.0,9.0,Foul_Tip
311,Wilson Ramos,"C,DH",NYM,207.0,314.0,53.0,46.0,37.0,46.0,25.0,0.0,BC-Mizusawa South Monkeyz
396,Danny Jansen,C,TOR,189.0,396.0,39.0,55.0,27.0,33.0,28.0,7.0,NaN
398,Francisco Cervelli,C,PIT,188.0,399.5,44.0,57.0,22.0,33.0,25.0,7.0,NaN
431,Tucker Barnhart,C,CIN,179.0,434.5,41.0,57.0,20.0,32.0,25.0,4.0,NaN


### OF

In [1430]:
outfield = all_players_owners[all_players_owners['Positions'].str.contains('OF|LF|RF|CF', regex=True)]
outfield = outfield[ ['Player', 'Positions', 'Team', 'SCORE', 'RANK', 'AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'Owner'] ]

In [1431]:
outfield[:25]

,Player,Positions,Team,SCORE,RANK,AVGS,BBKOS,HRS,RBIS,RS,SBS,Owner
0,Mike Trout,"CF,DH",LAA,505.0,1.0,86.0,100.0,90.0,81.0,97.0,51.0,MitchMoreland'sBattingAverage
1,Mookie Betts,"CF,RF",BOS,474.0,2.0,91.0,77.0,71.0,77.0,100.0,58.0,Tommy's Shinebox
3,J.D. Martinez,"LF,RF,DH",BOS,418.0,4.0,85.0,50.0,93.0,100.0,81.0,9.0,stroke me strokeme
4,Bryce Harper,"CF,RF",PHI,412.0,5.0,53.0,88.0,80.0,85.0,82.0,24.0,Regular Rubens
6,Christian Yelich,"LF,CF,RF",MIL,410.0,6.5,85.0,61.0,63.0,78.0,85.0,38.0,BC-Mizusawa South Monkeyz
11,Juan Soto,LF,WSH,393.0,12.0,72.0,80.0,68.0,80.0,77.0,16.0,Rakuten Golden Eagles
15,Aaron Judge,"RF,DH",NYY,383.0,16.0,51.0,63.0,90.0,79.0,84.0,16.0,Foul_Tip
19,Ronald Acuna,"LF,CF",ATL,379.0,19.5,64.0,42.0,71.0,72.0,79.0,51.0,Team Gorgeous
20,Giancarlo Stanton,"LF,RF,DH",NYY,379.0,19.5,53.0,44.0,100.0,94.0,81.0,7.0,Tommy's Shinebox
21,Charlie Blackmon,CF,COL,376.0,22.0,79.0,51.0,66.0,62.0,91.0,27.0,La Flama Blanca


In [1432]:
outfield[25:50]

,Player,Positions,Team,SCORE,RANK,AVGS,BBKOS,HRS,RBIS,RS,SBS,Owner
54,Aaron Hicks,CF,NYY,315.0,55.0,38.0,73.0,56.0,61.0,65.0,22.0,Low hanging fruit
58,Michael Brantley,"LF,DH",HOU,311.0,59.5,74.0,64.0,34.0,58.0,57.0,24.0,Regular Rubens
61,Nicholas Castellanos,RF,DET,311.0,59.5,68.0,40.0,59.0,74.0,63.0,7.0,MitchMoreland'sBattingAverage
62,Michael Conforto,"LF,CF,RF",NYM,309.0,64.5,38.0,60.0,68.0,72.0,64.0,7.0,BC-Mizusawa South Monkeyz
67,Justin Upton,"LF,DH",LAA,308.0,67.5,40.0,43.0,68.0,74.0,65.0,18.0,Martin Awaiting Klub
70,Joey Gallo,"1B,LF,CF,RF",TEX,306.0,71.0,11.0,44.0,95.0,78.0,67.0,11.0,stroke me strokeme
79,David Peralta,LF,ARI,298.0,79.5,67.0,47.0,54.0,60.0,59.0,11.0,Foul_Tip
80,Wil Myers,"3B,LF,RF",SD,297.0,83.5,39.0,43.0,56.0,60.0,57.0,42.0,BC-Mizusawa South Monkeyz
83,David Dahl,"LF,CF,RF",COL,297.0,83.5,58.0,34.0,59.0,67.0,55.0,24.0,Low hanging fruit
84,Ender Inciarte,CF,ATL,297.0,83.5,64.0,57.0,20.0,43.0,62.0,51.0,Rakuten Golden Eagles


In [1433]:
#outfield[50:75]

### SP

In [1434]:
starter = all_players_owners[all_players_owners['Positions'].str.contains('SP')]
starter = starter[ ['Player', 'Positions', 'Team', 'SCORE', 'RANK', 'ERAS', 'WHIPS', 'KS', 'WLS', 'SV+HDS', 'Owner'] ]

In [1435]:
starter[:25]

,Player,Positions,Team,SCORE,RANK,ERAS,WHIPS,KS,WLS,SV+HDS,Owner
12,Chris Sale,SP,BOS,388.0,13.0,97.0,99.0,92.0,100.0,0.0,Rakuten Golden Eagles
13,Max Scherzer,SP,WSH,385.0,14.0,92.0,100.0,100.0,93.0,0.0,Low hanging fruit
24,Jacob deGrom,SP,NYM,364.0,25.0,100.0,94.0,91.0,79.0,0.0,Team Gorgeous
25,Justin Verlander,SP,HOU,361.0,26.5,84.0,94.0,90.0,93.0,0.0,stroke me strokeme
32,Corey Kluber,SP,CLE,347.0,32.5,84.0,90.0,80.0,93.0,0.0,Tommy's Shinebox
50,Gerrit Cole,SP,HOU,322.0,51.0,77.0,79.0,87.0,79.0,0.0,Low hanging fruit
53,Carlos Carrasco,SP,CLE,317.0,53.5,78.0,81.0,79.0,79.0,0.0,stroke me strokeme
66,Blake Snell,SP,TB,308.0,67.5,80.0,71.0,78.0,79.0,0.0,Regular Rubens
68,Aaron Nola,SP,PHI,307.0,69.5,81.0,79.0,76.0,71.0,0.0,Walk Off
69,Trevor Bauer,SP,CLE,307.0,69.5,76.0,72.0,80.0,79.0,0.0,Rakuten Golden Eagles


In [1436]:
starter[25:50]

,Player,Positions,Team,SCORE,RANK,ERAS,WHIPS,KS,WLS,SV+HDS,Owner
163,Masahiro Tanaka,SP,NYY,251.0,164.0,60.0,69.0,58.0,64.0,0.0,Foul_Tip
164,Kyle Hendricks,SP,CHC,250.0,166.5,64.0,68.0,54.0,64.0,0.0,Rakuten Golden Eagles
165,Jose Berrios,SP,MIN,250.0,166.5,60.0,65.0,68.0,57.0,0.0,Martin Awaiting Klub
173,J.A. Happ,SP,NYY,248.0,173.0,60.0,65.0,59.0,64.0,0.0,Team Gorgeous
178,Chris Archer,SP,PIT,246.0,179.5,63.0,61.0,72.0,50.0,0.0,Martin Awaiting Klub
182,Mike Foltynewicz,SP,ATL,245.0,182.0,63.0,62.0,63.0,57.0,0.0,Tommy's Shinebox
183,Rich Hill,SP,LAD,244.0,185.0,66.0,68.0,53.0,57.0,0.0,Regular Rubens
184,Robbie Ray,SP,ARI,244.0,185.0,62.0,56.0,76.0,50.0,0.0,La Flama Blanca
185,Jameson Taillon,SP,PIT,244.0,185.0,67.0,66.0,61.0,50.0,0.0,Martin Awaiting Klub
193,German Marquez,SP,COL,242.0,191.5,60.0,60.0,72.0,50.0,0.0,Regular Rubens


### RP

In [1437]:
relief = all_players_owners[all_players_owners['Positions'].str.contains('RP')]
relief = relief[ ['Player', 'Positions', 'Team', 'SCORE', 'RANK', 'ERAS', 'WHIPS', 'KS', 'WLS', 'SV+HDS', 'Owner'] ]

In [1438]:
relief[:25]

,Player,Positions,Team,SCORE,RANK,ERAS,WHIPS,KS,WLS,SV+HDS,Owner
76,Edwin Diaz,RP,NYM,300.0,76.5,68.0,66.0,32.0,43.0,91.0,Foul_Tip
89,Josh Hader,RP,MIL,294.0,90.5,67.0,64.0,40.0,43.0,80.0,Walk Off
101,Seranthony Dominguez,"SP,RP",PHI,281.0,102.5,59.0,57.0,24.0,43.0,98.0,NaN
103,Andrew Miller,RP,STL,281.0,102.5,59.0,57.0,22.0,43.0,100.0,NaN
106,Brad Hand,RP,CLE,279.0,106.5,61.0,58.0,28.0,43.0,89.0,Low hanging fruit
109,Kenley Jansen,RP,LAD,278.0,110.0,61.0,64.0,24.0,36.0,93.0,Rakuten Golden Eagles
115,Aroldis Chapman,RP,NYY,275.0,116.5,62.0,58.0,28.0,43.0,84.0,MitchMoreland'sBattingAverage
117,Blake Treinen,RP,OAK,274.0,118.5,66.0,59.0,24.0,43.0,82.0,Foul_Tip
132,Adam Ottavino,RP,NYY,266.0,132.5,57.0,53.0,26.0,43.0,87.0,NaN
137,Felipe Vazquez,RP,PIT,262.0,138.5,63.0,56.0,25.0,36.0,82.0,Rakuten Golden Eagles


## My Team

In [1439]:
my_team = all_players_owners[ all_players_owners['Owner'] == 'Tommy\'s Shinebox' ]
my_team

,Player,Positions,Team,SCORE,RANK,AVGS,BBKOS,HRS,RBIS,RS,SBS,ERAS,WHIPS,KS,WLS,SV+HDS,Owner
1,Mookie Betts,"CF,RF",BOS,474.0,2.0,91.0,77.0,71.0,77.0,100.0,58.0,NaN,NaN,NaN,NaN,NaN,Tommy's Shinebox
20,Giancarlo Stanton,"LF,RF,DH",NYY,379.0,19.5,53.0,44.0,100.0,94.0,81.0,7.0,NaN,NaN,NaN,NaN,NaN,Tommy's Shinebox
32,Corey Kluber,SP,CLE,347.0,32.5,NaN,NaN,NaN,NaN,NaN,NaN,84.0,90.0,80.0,93.0,0.0,Tommy's Shinebox
34,Nelson Cruz,DH,MIN,343.0,34.5,56.0,51.0,83.0,87.0,64.0,2.0,NaN,NaN,NaN,NaN,NaN,Tommy's Shinebox
41,Lorenzo Cain,CF,MIL,327.0,42.0,74.0,63.0,29.0,39.0,69.0,53.0,NaN,NaN,NaN,NaN,NaN,Tommy's Shinebox
47,Carlos Santana,"1B,3B",CLE,324.0,47.0,37.0,92.0,54.0,70.0,64.0,7.0,NaN,NaN,NaN,NaN,NaN,Tommy's Shinebox
73,Mike Moustakas,"3B,DH",MIL,303.0,73.5,45.0,51.0,73.0,76.0,54.0,4.0,NaN,NaN,NaN,NaN,NaN,Tommy's Shinebox
74,Eric Hosmer,1B,SD,301.0,75.0,56.0,57.0,49.0,65.0,61.0,13.0,NaN,NaN,NaN,NaN,NaN,Tommy's Shinebox
92,Rougned Odor,"2B,DH",TEX,291.0,93.0,36.0,37.0,59.0,64.0,64.0,31.0,NaN,NaN,NaN,NaN,NaN,Tommy's Shinebox
112,Patrick Corbin,SP,WSH,276.0,114.0,NaN,NaN,NaN,NaN,NaN,NaN,72.0,68.0,72.0,64.0,0.0,Tommy's Shinebox


In [1440]:
round(my_team[ ['AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'ERAS', 'WHIPS', 'KS', 'WLS', 'SV+HDS'] ].mean())

AVGS      56.0
BBKOS     56.0
HRS       58.0
RBIS      65.0
RS        64.0
SBS       26.0
ERAS      69.0
WHIPS     68.0
KS        61.0
WLS       63.0
SV+HDS    14.0
dtype: float64

## Roto Analysis

In [1441]:
cols = ['Team', 'Count', 'AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'ERAS', 'WHIPS', 'KS', 'WLS', 'SV+HDS' ]
roto_score = pd.DataFrame(columns = cols)
for team in pd.unique(all_players_owners['Owner'].dropna()):
    roster = all_players_owners[ all_players_owners['Owner'] == team ]
    row = []
    row.append(team)
    row.append(len(roster))
    for stat in ['AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'ERAS', 'WHIPS', 'KS', 'WLS', 'SV+HDS']:
        if roster[stat].isnull().all():
            row.append(0)
        else:
            row.append(round(roster[stat].mean()))
    row = tuple(row)
    rosterdf = pd.DataFrame([row], columns=cols)
    roto_score = roto_score.append(rosterdf, ignore_index=True)

In [1442]:
roto_score

,Team,Count,AVGS,BBKOS,HRS,RBIS,RS,SBS,ERAS,WHIPS,KS,WLS,SV+HDS
0,MitchMoreland'sBattingAverage,14,59,58,60,69,62,15,67,66,44,53,31
1,Tommy's Shinebox,15,56,56,58,65,64,26,69,68,61,63,14
2,BC-Mizusawa South Monkeyz,13,54,57,53,66,64,28,59,59,57,50,0
3,stroke me strokeme,13,59,51,62,72,66,15,70,73,64,66,18
4,Regular Rubens,13,58,63,48,61,65,32,66,64,58,57,10
5,Foul_Tip,13,56,49,54,62,61,27,64,65,44,54,43
6,Martin Awaiting Klub,13,59,59,54,63,67,26,64,66,67,52,0
7,Walk Off,14,59,50,54,65,64,24,70,68,58,61,16
8,La Flama Blanca,12,54,60,57,62,69,20,59,61,58,54,0
9,Rakuten Golden Eagles,12,62,70,51,63,71,25,69,69,56,58,29


In [1443]:
cols = ['AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'ERAS', 'WHIPS', 'KS', 'WLS', 'SV+HDS' ]
for col in cols:
    roto_score[col] = roto_score[col].rank(method='average', na_option='keep', ascending=True, pct=False)
roto_score['Score'] = roto_score[cols].sum(axis=1)

In [1444]:
roto_score.sort_values(['Score'], ascending=False)

,Team,Count,AVGS,BBKOS,HRS,RBIS,RS,SBS,ERAS,WHIPS,KS,WLS,SV+HDS,Score
3,stroke me strokeme,13,8.5,4.0,11.0,11.0,8.5,1.5,10.5,11.5,11.0,11.0,8.0,96.5
11,Team Gorgeous,13,3.0,1.0,12.0,12.0,8.5,8.0,12.0,11.5,10.0,12.0,5.0,95.0
9,Rakuten Golden Eagles,12,12.0,12.0,2.0,4.5,12.0,6.0,8.0,9.0,3.0,7.0,9.0,84.5
10,Low hanging fruit,14,11.0,5.5,7.0,9.0,1.0,3.0,8.0,10.0,6.5,8.0,11.0,80.0
1,Tommy's Shinebox,15,4.5,5.5,9.0,6.5,5.0,8.0,8.0,7.5,9.0,10.0,6.0,79.0
7,Walk Off,14,8.5,3.0,5.0,6.5,5.0,5.0,10.5,7.5,6.5,9.0,7.0,73.5
6,Martin Awaiting Klub,13,8.5,9.0,5.0,4.5,10.0,8.0,3.5,5.5,12.0,2.0,2.0,70.0
0,MitchMoreland'sBattingAverage,14,8.5,8.0,10.0,10.0,3.0,1.5,6.0,5.5,1.5,3.0,10.0,67.0
4,Regular Rubens,13,6.0,11.0,1.0,1.0,7.0,12.0,5.0,3.0,6.5,6.0,4.0,62.5
8,La Flama Blanca,12,1.5,10.0,8.0,2.5,11.0,4.0,1.5,2.0,6.5,4.5,2.0,53.5


In [1445]:
roto_score_adj = roto_score
del roto_score_adj['SV+HDS']
cols = ['AVGS', 'BBKOS', 'HRS', 'RBIS', 'RS', 'SBS', 'ERAS', 'WHIPS', 'KS', 'WLS' ]
roto_score_adj['Score'] = roto_score[cols].sum(axis=1)

In [1446]:
roto_score_adj.sort_values(['Score'], ascending=False)

,Team,Count,AVGS,BBKOS,HRS,RBIS,RS,SBS,ERAS,WHIPS,KS,WLS,Score
11,Team Gorgeous,13,3.0,1.0,12.0,12.0,8.5,8.0,12.0,11.5,10.0,12.0,90.0
3,stroke me strokeme,13,8.5,4.0,11.0,11.0,8.5,1.5,10.5,11.5,11.0,11.0,88.5
9,Rakuten Golden Eagles,12,12.0,12.0,2.0,4.5,12.0,6.0,8.0,9.0,3.0,7.0,75.5
1,Tommy's Shinebox,15,4.5,5.5,9.0,6.5,5.0,8.0,8.0,7.5,9.0,10.0,73.0
10,Low hanging fruit,14,11.0,5.5,7.0,9.0,1.0,3.0,8.0,10.0,6.5,8.0,69.0
6,Martin Awaiting Klub,13,8.5,9.0,5.0,4.5,10.0,8.0,3.5,5.5,12.0,2.0,68.0
7,Walk Off,14,8.5,3.0,5.0,6.5,5.0,5.0,10.5,7.5,6.5,9.0,66.5
4,Regular Rubens,13,6.0,11.0,1.0,1.0,7.0,12.0,5.0,3.0,6.5,6.0,58.5
0,MitchMoreland'sBattingAverage,14,8.5,8.0,10.0,10.0,3.0,1.5,6.0,5.5,1.5,3.0,57.0
8,La Flama Blanca,12,1.5,10.0,8.0,2.5,11.0,4.0,1.5,2.0,6.5,4.5,51.5
